<p float="left">
  <img src="images/arrow_logo.jpg" alt="Arrow Logo" width="200" height="100" align="left"/>
  <img src="images/adi_logo.png" alt="ADI Logo" width="200" height="100" align="right"/>
</p>

<hr style="height:5px; text-align:bottom">

# Run The Cells Below
Run the cells below in sequence from top to bottom. Upon running the cell which initializes the required classes, a control widget and a pop-up containing data plots will appear.

To execute a cell:
1. Select the cell you wish to run by clicking the white space surrounding it. A blue border will appear around the cell.

    <img src="images/jupyter_cell_selected.PNG" alt="Jupyter Cell Selected"/>

2. Click the <strong>Run</strong> button at the top of the notebook to run the selected cell. Alternatively, you can use the CTRL+Enter keyboard combination to run the selected cell.
    - You can run the cells with markdown code in them, they have no effect on the program
    
3. After the cell has been run, a number indicating the current execution step will be inserted between the brackets on the left side of the cell.

    <img src="images/jupyter_cell_executed.PNG" alt="Jupyter Cell Executed"/>
    
    - Occasionally, after running the cell an asterisk may appear instead of a number, this indicates the cell is busy. Simply run the cell again to force/continue code execution.

    <img src="images/jupyter_cell_busy.PNG" alt="Jupyter Cell Busy"/>


If you make a mistake or run into problems, click <strong>Kernel -> Restart & Clear Output</strong> in the toolbar at the top of the page and start again. This clears the kernel of any variables or functions that may have been created in the previous program.

<hr style="height:1px">

## Modifying the AD7124 On-Chip Registers

1.Restart the kernel and clear the output by clicking <strong>Kernel -> Restart & Clear Output</strong> in the toolbar at the top of the page. This clears the kernel of any variables or functions that may have been created in the previous program.

2.Create a copy of the ad7124_default_config.py file and name it 'ad7124_custom_config.py'. This newly created file will be used for all of the custom modification you would like to make.

In [ ]:
import shutil

shutil.copy2('ad7124_default_config.py', 'ad7124_custom_config.py')

3.Modify the default values for the AD7124 registers as needed, following the  [AD7124 datasheet](https://www.analog.com/media/en/technical-documentation/data-sheets/ad7124-8.pdf) for reference

##### A. First we need to import the Serial and Data modules required for the demo. We will also set the Matplotlib backend to provide a pop-up window for our data plots

In [ ]:
from safari_serial import Serial
from safari_data import Data

# Set the TkAgg backend for matplotlib
%matplotlib tk

4.Next we need to update the config file, this is done by simply passing the name of the custom configuration file to the Serial class when it is initialized. This is done for you in the cell below.

<span style='color:red'><strong>You should modify the 'channels' data structure according to your AD7124 configuration and application needs. The 'channels' structure used below is for reference only</strong></span>

To edit the cell click inside it. The border will turn green.

After you run the cell you will be prompted to select the COM port that is connected to the Meerkat. Please refer to the [Identify COM Port](https://github.com/ArrowElectronics/arrow-safari/wiki/Running-Demo-in-Jupyter-Notebook#2-identify-com-port) section of the wiki for instructions.

<span style='color:red'><strong>Please note that the configuration file will be written to the board once the connect button is clicked.</strong></span>

In [ ]:
from safari_serial import Serial
from safari_data import Data

# Set the TkAgg backend for matplotlib
%matplotlib tk

# Add entries for each enabled channel. For example if you only enabled channel 5 in your custom configuration:
channels = {"0":{}}

#default fields for each channel
for channel in channels:
    channels[channel]["max_elements"] = 300
    channels[channel]["timestamps"]=[]
    channels[channel]["voltages"]=[]
    channels[channel]["values"]=[]
    channels[channel]["axis"] = None
    channels[channel]["annotation"] = None
    channels[channel]["annotation_visible"] = False
    channels[channel]["annotation_text"] = ""
    channels[channel]["line"] = None
    channels[channel]["annot_ax_xy"] = None
    channels[channel]["axes_color"] = 'k'
    channels[channel]["plot_title"] = f"Channel {channel}"
    channels[channel]["y_label"] = ""
    channels[channel]["y_label_converted"] = ""
    channels[channel]["voltage_format_func"] = None
    channels[channel]["value_conversion_func"] = None
    
# Function for converting channel 0 voltage to an accelerometer value
def convert_accelerometer(voltage, code):
    del code
    
    vcc = 3.3
    return round((voltage - vcc / 2) * 1 / 0.640, 3)
    
# Sample Configuration Channel 0 - Configure according to your application
channels["0"]["axes_color"] = 'tab:blue'
channels["0"]["plot_title"] = "Channel 0"
channels["0"]["y_label"] = 'Voltage (V)'
channels["0"]["y_label_converted"] = 'Accel. (g)'
channels["0"]["value_conversion_func"] = convert_accelerometer

serial = Serial(channels, 'ad7124_custom_config.py')
data = Data(channels)

5.It is good practice to update the AD7124 python driver (ad7124.py) as well as the data collection application (safari.py) on the Meerkat as well. Execute the below cells <strong>AFTER</strong> successfully connecting the serial port.

In [ ]:
serial.write_file_to_meerkat("ad7124.py", "ad7124.py")

In [ ]:
serial.write_file_to_meerkat("safari.py", "safari.py")

6.Once you are finished running the demo, shutdown the kernel by clicking <strong>Kernel -> Shutdown</strong> in the toolbar at the top of the page.